In [1]:
import numpy as np
import time

from mlflow.tracking import MlflowClient
import mlflow

In [2]:
import os
os.environ['MLFLOW_TRACKING_USERNAME'] = "cuong"
os.environ['MLFLOW_TRACKING_PASSWORD'] = "ds@cuong1"
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5005"
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://172.19.164.128:9000"
os.environ['RAY_API_ENDPOINT_URL'] = "http://127.0.0.1:8000"
os.environ['AWS_ACCESS_KEY_ID'] = "zL8jRR2H28UZ6k2ZSTGe"
os.environ['AWS_SECRET_ACCESS_KEY'] = "0g23JxsUWV9LovRCuXZbazkMwXgXpEd7kPuKbCxc" 

In [3]:
from mlflow.deployments import get_deploy_client
target_uri = 'ray-serve'
plugin = get_deploy_client(target_uri)

/mnt/e/setup/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-24 07:59:14,890	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-08-24 07:59:15,198	WARNING api.py:346 -- The default value for `max_ongoing_requests` has changed from 100 to 5 in Ray 2.32.0.
2024-08-24 07:59:15,199	WARNING api.py:397 -- The default value for `max_ongoing_requests` has changed from 100 to 5 in Ray 2.32.0.
2024-08-24 07:59:15,207	INFO worker.py:1596 -- Connecting to existing Ray cluster at address: 172.19.164.128:6379...
2024-08-24 07:59:15,215	INFO worker.py:1772 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
(ServeController pid=15371) INFO 2024-08-24 07:59:15,420 controller 15371 deploym

In [4]:
plugin.create_deployment(
    name="xgboost_model:v1",
    model_uri="models:/xgboost_model/1",
    config={"num_replicas": 2})

The new client HTTP config differs from the existing one in the following fields: ['host']. The new HTTP config is ignored.
2024-08-24 07:59:15,329	INFO handle.py:126 -- Created DeploymentHandle 'dbk6jv0d' for Deployment(name='xgboost_model:v1', app='xgboost_model:v1').
2024-08-24 07:59:15,330	INFO handle.py:126 -- Created DeploymentHandle 'ks3kqtat' for Deployment(name='xgboost_model:v1', app='xgboost_model:v1').
2024-08-24 07:59:18,346	INFO handle.py:126 -- Created DeploymentHandle 'c6300jn3' for Deployment(name='xgboost_model:v1', app='xgboost_model:v1').
2024-08-24 07:59:18,347	INFO api.py:574 -- Deployed app 'xgboost_model:v1' successfully.


{'name': 'xgboost_model:v1',
 'config': {'num_replicas': 2},
 'flavor': 'python_function'}

In [5]:
plugin.list_deployments()

[{'name': 'app1',
  'info': ApplicationStatusOverview(status=<ApplicationStatus.RUNNING: 'RUNNING'>, message='', last_deployed_time_s=1724381299.0933373, deployments={'Translator': DeploymentStatusOverview(status=<DeploymentStatus.HEALTHY: 'HEALTHY'>, status_trigger=<DeploymentStatusTrigger.CONFIG_UPDATE_COMPLETED: 'CONFIG_UPDATE_COMPLETED'>, replica_states={'RUNNING': 1}, message=''), 'Summarizer': DeploymentStatusOverview(status=<DeploymentStatus.HEALTHY: 'HEALTHY'>, status_trigger=<DeploymentStatusTrigger.CONFIG_UPDATE_COMPLETED: 'CONFIG_UPDATE_COMPLETED'>, replica_states={'RUNNING': 1}, message='')})},
 {'name': 'xgboost_model:v1',
  'info': ApplicationStatusOverview(status=<ApplicationStatus.RUNNING: 'RUNNING'>, message='', last_deployed_time_s=1724461155.3343494, deployments={'xgboost_model:v1': DeploymentStatusOverview(status=<DeploymentStatus.HEALTHY: 'HEALTHY'>, status_trigger=<DeploymentStatusTrigger.CONFIG_UPDATE_COMPLETED: 'CONFIG_UPDATE_COMPLETED'>, replica_states={'RUNNIN

# Read the data
Read the white wine quality and red wine quality CSV datasets and merge them into a single DataFrame.

In [6]:
# trick
# You can copy/paste the url and change 2 things:

# Remove "blob"
# Replace github.com by raw.githubusercontent.com
# For instance this link:

import pandas as pd 
red_wine = pd.read_table("https://raw.githubusercontent.com//aniruddhachoudhury/Red-Wine-Quality/master/winequality-red.csv", sep=",")
white_wine = pd.read_table("https://raw.githubusercontent.com//stedy/Machine-Learning-with-R-datasets/master/winequality-white.csv", sep=",")


In [7]:
red_wine['is_red'] = 1
white_wine['is_red'] = 0

data = pd.concat([red_wine, white_wine], axis=0)

# Remove spaces from column names
data.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

Looks like quality scores are normally distributed between 3 and 9.

Define a wine as high quality if it has quality >= 7.

In [8]:
high_quality = (data.quality >= 7).astype(int)
data.quality = high_quality

Box plots are useful for identifying correlations between features and a binary label. Create box plots for each feature to compare high-quality and low-quality wines. Significant differences in the box plots indicate good predictors of quality.

# Prepare the dataset to train a baseline model
Split the input data into 3 sets:

Train (60% of the dataset used to train the model)
Validation (20% of the dataset used to tune the hyperparameters)
Test (20% of the dataset used to report the true performance of the model on an unseen dataset)

In [9]:
from sklearn.model_selection import train_test_split

X = data.drop(["quality"], axis=1)
y = data.quality

# Split out the training data
X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.6, random_state=123)

# Split the remaining data equally into validation and test
X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, test_size=0.5, random_state=123)

In [10]:
# File name: composed_client.py
import requests
import json
body = {
    "dataframe": X_val.iloc[:20].to_json(orient="records"),
     #"schema": json.dumps(X_val.dtypes.to_dict(), default=str)
    
}
response = requests.post("http://127.0.0.1:8000/xgboost_model:v1", json=body)
french_text = response.text

print(french_text)

[0.020031269639730453,0.0014225119957700372,0.004350167699158192,0.7120213508605957,0.011486390605568886,0.045975130051374435,0.016866028308868408,0.06676547229290009,0.003596778027713299,0.006428544409573078,0.03146544471383095,0.2901235520839691,0.04730091243982315,0.004035182762891054,0.002892951713874936,0.6780819296836853,0.004918837919831276,0.6043729186058044,0.5013774037361145,0.8902697563171387]


In [11]:
plugin.predict(deployment_name="xgboost_model:v1", df=X_val)

'[0.020031269639730453,0.0014225119957700372,0.004350167699158192,0.7120213508605957,0.011486390605568886,0.045975130051374435,0.016866028308868408,0.06676547229290009,0.003596778027713299,0.006428544409573078,0.03146544471383095,0.2901235520839691,0.04730091243982315,0.004035182762891054,0.002892951713874936,0.6780819296836853,0.004918837919831276,0.6043729186058044,0.5013774037361145,0.8902697563171387,0.8440412878990173,0.0014654246624559164,0.010936100035905838,0.0651741698384285,0.6417548060417175,0.012345516122877598,0.09265454113483429,0.07240229099988937,0.012441970407962799,0.08253736793994904,0.017562389373779297,0.052249010652303696,0.023848263546824455,0.08605757355690002,0.03282793238759041,0.2080508917570114,0.002416325733065605,0.000939252320677042,0.04745626449584961,0.5615859031677246,0.0009137691813521087,0.009541105479001999,0.23492971062660217,0.017878830432891846,0.15561741590499878,0.022787196561694145,0.876081109046936,0.26137417554855347,0.0025792832020670176,0.

In [12]:
plugin.delete_deployment(name="xgboost_model:v1")